**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer__

The function act implements an $\epsilon$-greedy policy during training which is defined as 
\begin{equation}
    \pi(s)=\left\{
    \begin{aligned}
    \mathrm{random ~action~} & \quad \mathrm{with ~probability ~} \epsilon \\
    \mathrm{the ~best ~learned ~action ~so ~far} & \quad \mathrm{with ~probability ~} 1 - \epsilon
    \end{aligned}
    \right.
\end{equation}
and during testing it takes the best learned action.

The ```epsilon``` is essential because it determines the tradeoff between exploration and exploitation. When $\epsilon$ is too large, we explore too much but do not exploit what we have already learned. When $\epsilon$ is too small, we do not do enough exploration to learn the best action.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Answer__

The array ```position``` stores the current position of the rat, with value 1 at the current position of the rat and value 0 at other positions and the margins are set to -1.

The array ```board``` stores the whole environment of the game, with value 0.5 at the positions of cheese (bonus) and value -1 at the positions of poison (malus) and value 0 if there is nothing.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # take random action in {0, 1, 2, 3}
        a = np.random.randint(4, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action in "test" mode (train = False)
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.5/14.0. Average score (-1.5)
Win/lose count 9.0/12.0. Average score (-2.25)
Win/lose count 9.5/12.0. Average score (-2.3333333333333335)
Win/lose count 10.5/19.0. Average score (-3.875)
Win/lose count 11.0/18.0. Average score (-4.5)
Win/lose count 8.5/13.0. Average score (-4.5)
Win/lose count 11.0/19.0. Average score (-5.0)
Win/lose count 12.5/9.0. Average score (-3.9375)
Win/lose count 10.5/14.0. Average score (-3.888888888888889)
Win/lose count 7.0/11.0. Average score (-3.9)
Final score: -3.9


__Remark__

The random action produces a negative score as the lose of a poison cell (1) is larger than the win of a cheese cell (0.5).

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer__

As mentioned at the beggining of this project, the definition of the $Q$-function is 
\begin{equation}
    Q^\pi (s, a) = \mathbb{E}_{p^\pi}\left[\sum_{t=0}^T \gamma^t r(s_t, a_t) ~|~ s_0 = s, a_0 = a \right]
\end{equation}
Here $T = \infty$, then we have 
\begin{equation}
    \begin{split}
        Q^\pi (s, a) &= \mathbb{E}_{p^\pi}\left[r(s_0, a_0) + \sum_{t=1}^\infty \gamma^t r(s_t, a_t) ~|~ s_0 = s, a_0 = a \right] \\
                    &= r(s, a) + \mathbb{E}_{p^\pi} \left[\sum_{t=1}^\infty \gamma^t r(s_t, a_t) ~|~ s_0 = s, a_0 = a \right] \\
                    &= r(s, a) + \gamma \sum_{s^\prime, a^\prime} p(s^\prime ~|~ s, a)\pi(a^\prime~|~s^\prime)~ \mathbb{E}_{p^\pi} \left[\sum_{t=1}^\infty \gamma^{t-1} r(s_t, a_t) ~|~ s_1 = s^\prime, a_1 = a^\prime \right] \\
                    &= r(s, a) + \gamma \mathbb{E}_{(s^\prime, a^\prime)\sim p(\cdot |s, a) \pi(\cdot |s^\prime)} \left[Q^\pi (s^\prime, a^\prime) \right] \\
                    &= \mathbb{E}_{(s^\prime, a^\prime)\sim p(\cdot |s, a) \pi(\cdot |s^\prime)} \left[r(s, a) + \gamma Q^\pi (s^\prime, a^\prime) \right]
    \end{split}
\end{equation}
which is exactly that we want to show.

For any policy $\pi = (a^\prime, \pi^\prime)$, we have 
\begin{equation}
    \begin{split}
        Q^*(s, a) &= \max_{\pi} Q^\pi (s, a) \\
                &= \max_{a^\prime, \pi^\prime} \mathbb{E}_{s^\prime\sim p(\cdot |s, a)} \left[r(s, a) + \gamma Q^{\pi^\prime} (s^\prime, a^\prime) \right] \\
                &= \max_{a^\prime} \mathbb{E}_{s^\prime\sim p(\cdot |s, a)} \left[r(s, a) + \gamma \max_{\pi^\prime} Q^{\pi^\prime} (s^\prime, a^\prime) \right] \\
                &= \max_{a^\prime} \mathbb{E}_{s^\prime\sim p(\cdot |s, a)} \left[r(s, a) + \gamma Q^* (s^\prime, a^\prime) \right] \\
                &= \mathbb{E}_{s^\prime\sim p(\cdot |s, a)} \left[r(s, a) + \gamma \max_{a^\prime} Q^* (s^\prime, a^\prime) \right]
    \end{split}
\end{equation}
which is that we want to show.

Finally, we can estimate the new $Q$-function by 
\begin{equation}
    Q^{estim}(s, a, \theta) = \mathbb{E}_{s^\prime\sim p(\cdot |s, a)} \left[r(s, a) + \gamma \max_{a^\prime} Q (s^\prime, a^\prime, \theta) \right]
\end{equation}
Then we can use the quadratic loss function 
\begin{equation}
    \begin{split}
        \mathcal{L}(\theta) &= ||Q^{estim}(s, a, \theta) - Q(s, a, \theta)||^2 \\
                &= ||\mathbb{E}_{s^\prime\sim p(\cdot |s, a)} \left[r(s, a) + \gamma \max_{a^\prime} Q (s^\prime, a^\prime, \theta) \right] - Q(s, a, \theta)||^2 \\
                &= \mathbb{E}_{s^\prime\sim p(\cdot |s, a)} \left[ ||r(s, a) + \gamma \max_{a^\prime} Q (s^\prime, a^\prime, \theta) - Q(s, a, \theta)||^2\right]
    \end{split}
\end{equation}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # if the max memory size already reached, pop the oldest one and register the new one
        if len(self.memory) == self.max_memory:
            self.memory.pop(0)
            self.memory.append(m)
        # else simply register the new one
        else:
            self.memory.append(m)

    def random_access(self):
        # randomly take a sample in the memory
        idx = np.random.randint(len(self.memory), size=1)[0]
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        # print also 'Average score' to see if the agent learns to win
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | Average score ({})"
              .format(e, epoch, loss, win, lose, win-lose, score/(1+e)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # predict the Q function by the current model and take the action which maximizes the Q function
        q = self.model.predict(s.reshape((1, 5, 5, self.n_state)))
        a = np.argmax(q)
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # randomly take a sample from memory
            m = self.memory.random_access()
            s = m[0].reshape((1, 5, 5, self.n_state))
            n_s = m[1].reshape((1, 5, 5, self.n_state))
            a = m[2]
            r = m[3]
            game_over = m[4]
            # store the state as a sample in the batch and predict the Q function by the current model
            input_states[i] = s
            target_q[i] = self.model.predict(s)
            
            # change target_q[i, a] to the estimated value in the two cases whether the game is over
            if game_over:
                ######## FILL IN
                target_q[i, a] = r
            else:
                ######## FILL IN
                target_q[i, a] = r + self.discount * np.max(self.model.predict(n_s))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        # design a fully connected network with two hidden layers of 32 and 16 neurons
        ####### FILL IN
        model = keras.Sequential()
        model.add(keras.layers.Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(32))
        model.add(Activation('relu'))
        model.add(Dense(16))
        model.add(Activation('relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [12]:
# reset the parameters for training
temperature = 0.3 # one can set temperature smaller or larger to see its effect
epochs_train = 60 # one can take more epochs for training in order to learn more
epochs_test = 20
lr = 0.1
epsilon = 0.1
memory_size = 2000
batch_size = 32

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_FC(size, lr=lr, epsilon = epsilon, memory_size=memory_size, batch_size = batch_size)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/060 | Loss 0.0030 | Win/lose count 2.5/5.0 (-2.5) | Average score (-2.5)
Epoch 001/060 | Loss 0.0111 | Win/lose count 1.0/2.0 (-1.0) | Average score (-1.75)
Epoch 002/060 | Loss 0.0061 | Win/lose count 3.0/5.0 (-2.0) | Average score (-1.8333333333333333)
Epoch 003/060 | Loss 0.0200 | Win/lose count 5.0/7.0 (-2.0) | Average score (-1.875)
Epoch 004/060 | Loss 0.0041 | Win/lose count 3.5/6.0 (-2.5) | Average score (-2.0)
Epoch 005/060 | Loss 0.0081 | Win/lose count 1.0/1.0 (0.0) | Average score (-1.6666666666666667)
Epoch 006/060 | Loss 0.0106 | Win/lose count 3.5/3.0 (0.5) | Average score (-1.3571428571428572)
Epoch 007/060 | Loss 0.0086 | Win/lose count 5.5/12.0 (-6.5) | Average score (-2.0)
Epoch 008/060 | Loss 0.0078 | Win/lose count 1.5/4.0 (-2.5) | Average score (-2.0555555555555554)
Epoch 009/060 | Loss 0.0157 | Win/lose count 5.5/3.0 (2.5) | Average score (-1.6)
Epoch 010/060 | Loss 0.0138 | Win/lose count 4.5/1.0 (3.5) | Average score (-1.1363636363636365)
Epoch 011/06

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # design a CNN network with 2 conv layers of 16 and 32 filters and 1 fully connected layer
        
        ###### FILL IN
        model = keras.Sequential()
        model.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
        model.add(keras.layers.Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=lr, epsilon = epsilon, memory_size=memory_size, batch_size = batch_size)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/060 | Loss 0.0167 | Win/lose count 4.5/8.0 (-3.5) | Average score (-3.5)
Epoch 001/060 | Loss 0.0099 | Win/lose count 4.5/4.0 (0.5) | Average score (-1.5)
Epoch 002/060 | Loss 0.0080 | Win/lose count 3.0/3.0 (0.0) | Average score (-1.0)
Epoch 003/060 | Loss 0.0061 | Win/lose count 3.5/7.0 (-3.5) | Average score (-1.625)
Epoch 004/060 | Loss 0.0102 | Win/lose count 2.5/3.0 (-0.5) | Average score (-1.4)
Epoch 005/060 | Loss 0.0020 | Win/lose count 5.0/2.0 (3.0) | Average score (-0.6666666666666666)
Epoch 006/060 | Loss 0.0039 | Win/lose count 2.0/4.0 (-2.0) | Average score (-0.8571428571428571)
Epoch 007/060 | Loss 0.0059 | Win/lose count 3.0/2.0 (1.0) | Average score (-0.625)
Epoch 008/060 | Loss 0.0037 | Win/lose count 6.0/1.0 (5.0) | Average score (0.0)
Epoch 009/060 | Loss 0.0030 | Win/lose count 4.0/2.0 (2.0) | Average score (0.2)
Epoch 010/060 | Loss 0.0019 | Win/lose count 5.5/1.0 (4.5) | Average score (0.5909090909090909)
Epoch 011/060 | Loss 0.0088 | Win/lose count 3.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [16]:
env = Environment(grid_size=size, max_time=T,temperature=temperature)
agent_cnn = DQN_CNN(size, lr=lr, epsilon = epsilon, memory_size=memory_size, batch_size = batch_size)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=lr, epsilon = epsilon, memory_size=memory_size, batch_size = batch_size)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.0/0. Average score (3.0)
Win/lose count 11.5/1.0. Average score (6.75)
Win/lose count 8.0/0. Average score (7.166666666666667)
Win/lose count 5.0/1.0. Average score (6.375)
Win/lose count 6.0/0. Average score (6.3)
Win/lose count 2.5/0. Average score (5.666666666666667)
Win/lose count 1.5/0. Average score (5.071428571428571)
Win/lose count 4.5/0. Average score (5.0)
Win/lose count 2.0/0. Average score (4.666666666666667)
Win/lose count 16.5/0. Average score (5.85)
Win/lose count 2.5/0. Average score (5.545454545454546)
Win/lose count 1.0/0. Average score (5.166666666666667)
Win/lose count 2.0/0. Average score (4.923076923076923)
Win/lose count 5.5/0. Average score (4.964285714285714)
Win/lose count 16.0/0. Average score (5.7)
Win/lose count 7.5/0. Average score (5.8125)
Win/lose count 13.0/0. Average score (6.235294117647059)
Win/lose count 0.5/0. Average score (5.916666666666667)
Win/lose count 1.5/0. Average score (5.684210526315789)
Win/lose count 4.

In [17]:
HTML(display_videos('cnn_test10.mp4'))

In [18]:
HTML(display_videos('fc_test10.mp4'))

__Answer__

We can see that both two networks have learned to win the game, however the performance of CNN is better than that of Fully connected network (according to average score).

However, in both two cases, we can see that the rat does not explore enough the total map. It always gets trapped in a certain area.

The parameter ```temperature``` defines the probability of generating a cheese (or poison) at each cell. When ```temperature``` is small (e.g. 0.1), there is few cheese (or poison) cells. Thus it is difficult for the algorithm to learn the best policy. When ```temperature``` is large (e.g. 0.6), it is easier for the algorithm to learn the best policy.

More precisely, take the DQN_FC as an example, for the default value ```temperature = 0.3```, the average score starts to be positive after around 30 epochs. For ```temperature = 0.1```, it starts to be positive after around 50 epochs. For ```temperature = 0.6```, it starts to be positive at the beginning of epochs.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [19]:
def train_explore(agent, env, epoch, decay=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
            # decay the epsilon greedy parameter
            agent.set_epsilon(agent.epsilon * decay)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | Average score ({})"
              .format(e, epoch, loss, win, lose, win-lose, score/(1+e)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        # store malus to positions in an array
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        """The default value of train is False so that we do not need to modify the test function."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        # if in the "train" mode, the reward will be reduced by the value of the malus of this position
        # so if the position is already visited, the reward will be smaller than those cells that are not visited
        if train:
            reward -= self.malus_position[self.x, self.y]
            
        # we add 0.1 to the malus of the visited position
        self.malus_position[self.x, self.y] += 0.1
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # add malus_position information to the state
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1), 
                        self.board.reshape(self.grid_size, self.grid_size,1), 
                        self.position.reshape(self.grid_size, self.grid_size,1)), axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        # reset the malus_position to all zeros
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1), 
                               self.board.reshape(self.grid_size, self.grid_size,1), 
                        self.position.reshape(self.grid_size, self.grid_size,1)), axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [32]:
# set parameters
decay = 0.95
epsilon = 1.0
temperature = 0.3

In [33]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=lr, epsilon = epsilon, memory_size=memory_size, batch_size = batch_size,n_state=3)
train_explore(agent, env, epochs_train, decay=decay, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/060 | Loss 0.0507 | Win/lose count 8.5/52.00000000000004 (-43.50000000000004) | Average score (-43.50000000000004)
Epoch 001/060 | Loss 0.0197 | Win/lose count 21.0/31.00000000000003 (-10.000000000000028) | Average score (-26.750000000000036)
Epoch 002/060 | Loss 0.0159 | Win/lose count 14.0/34.20000000000001 (-20.20000000000001) | Average score (-24.56666666666669)
Epoch 003/060 | Loss 0.0120 | Win/lose count 18.0/23.600000000000016 (-5.600000000000016) | Average score (-19.825000000000024)
Epoch 004/060 | Loss 0.0191 | Win/lose count 21.0/19.199999999999985 (1.800000000000015) | Average score (-15.500000000000018)
Epoch 005/060 | Loss 0.0353 | Win/lose count 19.5/38.20000000000002 (-18.700000000000017) | Average score (-16.03333333333335)
Epoch 006/060 | Loss 0.0096 | Win/lose count 22.5/20.599999999999998 (1.9000000000000021) | Average score (-13.471428571428586)
Epoch 007/060 | Loss 0.0239 | Win/lose count 23.5/16.399999999999988 (7.100000000000012) | Average score (-10.9

In [34]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 25.5/4.0. Average score (21.5)
Win/lose count 25.5/4.0. Average score (21.5)
Win/lose count 23.0/8.0. Average score (19.333333333333332)
Win/lose count 27.0/6.0. Average score (19.75)
Win/lose count 22.5/5.0. Average score (19.3)
Win/lose count 23.0/5.0. Average score (19.083333333333332)
Win/lose count 24.0/5.0. Average score (19.071428571428573)
Win/lose count 23.5/6.0. Average score (18.875)
Win/lose count 21.0/5.0. Average score (18.555555555555557)
Win/lose count 29.0/2.0. Average score (19.4)
Win/lose count 23.5/7.0. Average score (19.136363636363637)
Win/lose count 25.0/6.0. Average score (19.125)
Win/lose count 26.0/3.0. Average score (19.423076923076923)
Win/lose count 21.0/3.0. Average score (19.321428571428573)
Win/lose count 22.0/9.0. Average score (18.9)
Win/lose count 22.0/6.0. Average score (18.71875)
Win/lose count 24.0/6.0. Average score (18.676470588235293)
Win/lose count 23.5/8.0. Average score (18.5)
Win/lose count 23.0/8.0. Average score (18.31578947

__Remark__

We can see that the final score is larger than the original DQN_CNN and the rat can explore almost the total map and visit almost all cheese cells. One may add more epochs for training on order to get a better performance which means reduce the loses.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

First, we define a function to generate winning games by the expert DQN, which returns a memory which stores all (state, next_state, action, reward, game_over) generated by the expert DQN.

In [35]:
def GenerateWinningGames(agent, env, epochs, memory_size):
    # memory of winning games
    memory = Memory(memory_size)
    
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action in "test" mode (train = False)
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            
            m = [prev_state, state, action, reward, game_over]
            memory.remember(m)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        # Update stats
        score = score + win - lose
        
        if e % 10 == 0:
            print("Game {}: Win/lose count {}/{}. Average score ({})"
                  .format(e+1, win, lose, score/(1+e)))
    
    return memory

Then we define the agent Mimic_DQN which takes the memory generated by the expert DQN as a parameter. And during training, it gets random samples from this memory. Different from DQN, it does not update the memory by its own (state, action) experience as it just mimicks the DQN. And we take to CNN network for training. What's more we do not take exploring training but take the constant ```epsilon = 0.1```.

In [36]:
class Mimic_DQN(Agent):
    def __init__(self, grid_size, memory, epsilon = 0.1, batch_size = 16, n_state=2):
        super(Mimic_DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = memory
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # predict the Q function by the current model and take the action which maximizes the Q function
        q = self.model.predict(s.reshape((1, 5, 5, self.n_state)))
        a = np.argmax(q)
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
#         self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # randomly take a sample from memory
            m = self.memory.random_access()
            s = m[0].reshape((1, 5, 5, self.n_state))
            n_s = m[1].reshape((1, 5, 5, self.n_state))
            a = m[2]
            r = m[3]
            game_over = m[4]
            # store the state as a sample in the batch and predict the Q function by the current model
            input_states[i] = s
            target_q[i] = self.model.predict(s)
            
            # change target_q[i, a] to the estimated value in the two cases whether the game is over
            if game_over:
                ######## FILL IN
                target_q[i, a] = r
            else:
                ######## FILL IN
                target_q[i, a] = r + self.discount * np.max(self.model.predict(n_s))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class Mimic_DQN_CNN(Mimic_DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(Mimic_DQN_CNN, self).__init__(*args,**kwargs)
        
        # design a CNN network with 2 conv layers of 16 and 32 filters and 1 fully connected layer
        
        ###### FILL IN
        model = keras.Sequential()
        model.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
        model.add(keras.layers.Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [37]:
epochs_gen = 100
memory_size = T * epochs_gen
epsilon = 0.1

In [38]:
memory = GenerateWinningGames(agent, env, epochs_gen, memory_size)

Game 1: Win/lose count 24.0/6.0. Average score (18.0)
Game 11: Win/lose count 19.0/7.0. Average score (17.90909090909091)
Game 21: Win/lose count 24.0/4.0. Average score (18.547619047619047)
Game 31: Win/lose count 20.5/8.0. Average score (18.306451612903224)
Game 41: Win/lose count 24.0/1.0. Average score (18.451219512195124)
Game 51: Win/lose count 24.0/6.0. Average score (18.666666666666668)
Game 61: Win/lose count 28.0/8.0. Average score (18.5327868852459)
Game 71: Win/lose count 26.0/7.0. Average score (18.640845070422536)
Game 81: Win/lose count 23.5/8.0. Average score (18.27777777777778)
Game 91: Win/lose count 22.0/6.0. Average score (18.263736263736263)


In [39]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)
mimic_agent = Mimic_DQN_CNN(size, lr=lr, memory=memory, epsilon = epsilon, batch_size = batch_size,n_state=3)
train(mimic_agent, env, epochs_train, prefix='mimic_cnn_train_explore')
# train_explore(mimic_agent, env, epochs_train, decay=decay, prefix='mimic_cnn_train_explore')
HTML(display_videos('mimic_cnn_train_explore10.mp4'))

Epoch 000/060 | Loss 0.0113 | Win/lose count 7.5/10.0 (-2.5) | Average score (-2.5)
Epoch 001/060 | Loss 0.0173 | Win/lose count 9.0/6.0 (3.0) | Average score (0.25)
Epoch 002/060 | Loss 0.0664 | Win/lose count 9.5/6.0 (3.5) | Average score (1.3333333333333333)
Epoch 003/060 | Loss 0.0575 | Win/lose count 7.0/4.0 (3.0) | Average score (1.75)
Epoch 004/060 | Loss 0.0037 | Win/lose count 1.0/1.0 (0.0) | Average score (1.4)
Epoch 005/060 | Loss 0.0177 | Win/lose count 5.0/2.0 (3.0) | Average score (1.6666666666666667)
Epoch 006/060 | Loss 0.0084 | Win/lose count 14.0/2.0 (12.0) | Average score (3.142857142857143)
Epoch 007/060 | Loss 0.0023 | Win/lose count 4.0/0 (4.0) | Average score (3.25)
Epoch 008/060 | Loss 0.0104 | Win/lose count 8.5/1.0 (7.5) | Average score (3.7222222222222223)
Epoch 009/060 | Loss 0.0037 | Win/lose count 6.0/2.0 (4.0) | Average score (3.75)
Epoch 010/060 | Loss 0.0022 | Win/lose count 7.0/2.0 (5.0) | Average score (3.8636363636363638)
Epoch 011/060 | Loss 0.0015 

In [40]:
# Evaluation
test(mimic_agent,env,epochs_test,prefix='mimic_cnn_test_explore')
HTML(display_videos('mimic_cnn_test_explore10.mp4'))

Win/lose count 0/0. Average score (0.0)
Win/lose count 7.0/0. Average score (3.5)
Win/lose count 2.5/0. Average score (3.1666666666666665)
Win/lose count 5.0/1.0. Average score (3.375)
Win/lose count 4.0/0. Average score (3.5)
Win/lose count 5.0/0. Average score (3.75)
Win/lose count 3.5/0. Average score (3.7142857142857144)
Win/lose count 9.5/0. Average score (4.4375)
Win/lose count 9.0/0. Average score (4.944444444444445)
Win/lose count 3.0/0. Average score (4.75)
Win/lose count 2.5/0. Average score (4.545454545454546)
Win/lose count 1.0/0. Average score (4.25)
Win/lose count 2.0/0. Average score (4.076923076923077)
Win/lose count 5.5/0. Average score (4.178571428571429)
Win/lose count 1.0/0. Average score (3.966666666666667)
Win/lose count 8.0/0. Average score (4.21875)
Win/lose count 3.5/0. Average score (4.176470588235294)
Win/lose count 5.5/0. Average score (4.25)
Win/lose count 6.0/0. Average score (4.342105263157895)
Win/lose count 1.5/0. Average score (4.2)
Final score: 4.2


We can see that the final score of the imitation network is worse than the previous expert DQN and it also has the exploration issue. This is because the memory generated by the expert DQN does not register the malus of position and we do not update the memory by the imitation network's own experience, thus the imitation network can not get rid of the exploration issue.

***